In [173]:
import os
import glob
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
from python_speech_features import mfcc , logfbank
import librosa as lr
import os, glob, pickle
import librosa
from scipy import signal
import noisereduce as nr
from glob import glob
import librosa
get_ipython().magic('matplotlib inline')
#All the Required Packages and Libraies are installed.
import soundfile
from tensorflow.keras.layers import Conv2D,MaxPool2D, Flatten, LSTM
from keras.layers import Dropout,Dense,TimeDistributed
from keras.models import Sequential
from keras.utils import to_categorical 
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score


In [96]:
pip install python_speech_features

In [4]:
pip install noisereduce

  Created wheel for noisereduce: filename=noisereduce-1.1.0-cp37-none-any.whl size=7610 sha256=b8ccb7905ee18578114cef44b1dc48e04e1b6800425ba0c09b3fe2693c56ce41
  Stored in directory: /root/.cache/pip/wheels/a6/2c/70/f9ccb41280dcfbe3eaeb7930f913dd85394617f3d3136f29cc
Successfully built noisereduce


In [174]:
import os
os.listdir(path='/content/drive/MyDrive/speech_emotion_recognition')
def getListOfFiles(dirName):
    listOfFile=os.listdir(dirName)
    allFiles=list()
    for entry in listOfFile:
        fullPath=os.path.join(dirName, entry)
        if os.path.isdir(fullPath):
            allFiles=allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
    return allFiles

dirName = '/content/drive/MyDrive/speech_emotion_recognition'
listOfFiles = getListOfFiles(dirName)
len(listOfFiles)

1440

In [175]:
pip install SpeechRecognition

In [176]:
def envelope(y , rate, threshold):
    mask=[]
    y=pd.Series(y).apply(np.abs)
    y_mean = y.rolling(window=int(rate/10) ,  min_periods=1 , center = True).mean()
    for mean in y_mean:
        if mean>threshold:
            mask.append(True)
        else:
            mask.append(False)
    return mask

In [177]:
import glob,pickle
for file in tqdm(glob.glob(r'/content/drive/MyDrive/speech_emotion_recognition//**//*.wav')):
    file_name = os.path.basename(file)
    signal , rate = librosa.load(file, sr=16000)
    mask = envelope(signal,rate, 0.0005)
    wavfile.write(filename= r'/content/drive/MyDrive/clean_speech1//'+str(file_name), rate=rate,data=signal[mask])

100%|██████████| 1440/1440 [00:43<00:00, 33.13it/s]


In [178]:
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
        result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
        result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
        result=np.hstack((result, mel))
    return result

In [179]:
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}
#These are the emotions User wants to observe more :
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [181]:
from glob import glob
import os
import glob
def load_data(test_size=0.33):
    x,y=[],[]
    answer = 0
    for file in glob.glob(r'/content/drive/MyDrive/clean_speech1//*.wav'):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            answer += 1
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append([emotion,file_name])
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [184]:
import librosa
import numpy as np
x_train,x_test,y_trai,y_tes=load_data(test_size=0.25)
print(np.shape(x_train),np.shape(x_test), np.shape(y_trai),np.shape(y_tes))
y_test_map = np.array(y_tes).T
y_test = y_test_map[0]
test_filename = y_test_map[1]
y_train_map = np.array(y_trai).T
y_train = y_train_map[0]
train_filename = y_train_map[1]
print(np.shape(y_train),np.shape(y_test))
print(*test_filename,sep="\n")

(576, 180) (192, 180) (576, 2) (192, 2)
(576,) (192,)
03-01-02-01-01-02-12.wav
03-01-06-01-01-02-22.wav
03-01-03-01-02-02-04.wav
03-01-02-01-02-02-21.wav
03-01-02-01-02-01-04.wav
03-01-02-01-02-01-07.wav
03-01-07-01-02-02-13.wav
03-01-02-01-01-02-15.wav
03-01-02-01-02-02-07.wav
03-01-03-02-01-01-10.wav
03-01-06-01-02-01-19.wav
03-01-06-01-01-01-12.wav
03-01-02-01-02-02-09.wav
03-01-07-01-02-01-07.wav
03-01-07-02-02-01-06.wav
03-01-02-02-01-01-08.wav
03-01-02-01-01-02-24.wav
03-01-03-01-02-01-13.wav
03-01-07-01-02-02-05.wav
03-01-06-02-02-01-15.wav
03-01-02-01-02-01-21.wav
03-01-03-02-01-01-16.wav
03-01-03-01-01-02-17.wav
03-01-03-01-02-02-15.wav
03-01-07-01-01-02-19.wav
03-01-07-01-01-01-20.wav
03-01-07-01-02-01-22.wav
03-01-03-01-01-01-18.wav
03-01-03-01-01-02-06.wav
03-01-02-01-01-02-22.wav
03-01-03-02-02-02-07.wav
03-01-06-02-01-02-01.wav
03-01-03-02-02-01-03.wav
03-01-07-02-01-02-22.wav
03-01-03-02-01-02-07.wav
03-01-07-02-02-01-02.wav
03-01-06-01-01-01-09.wav
03-01-02-02-02-02-01.

In [185]:
print((x_train[0], x_test[0]))
#Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')


(array([-3.90218903e+02,  9.14859085e+01, -3.55143166e+00,  1.52801123e+01,
        1.85004282e+00, -1.53970661e+01, -1.61027737e+01, -1.58910446e+01,
       -1.32888260e+01, -8.25821996e-01, -1.81923599e+01,  8.07108879e+00,
       -1.52320471e+01, -3.84366798e+00, -1.00702915e+01, -1.11860037e+01,
        1.09612811e+00, -3.85793376e+00, -8.92042542e+00, -2.90964007e+00,
       -8.00875568e+00, -5.92047024e+00, -7.76378441e+00, -2.21119142e+00,
       -6.08377981e+00, -1.53390253e+00, -2.12451053e+00, -1.35359788e+00,
       -8.34882545e+00, -3.35070109e+00, -5.17236567e+00, -2.55349851e+00,
       -2.02528238e+00, -7.20508194e+00, -7.52446413e+00, -5.71129227e+00,
       -3.13732743e-01,  1.62567973e+00,  6.47740841e-01, -3.38669032e-01,
        5.54685891e-01,  6.08335018e-01,  6.64716780e-01,  6.33815527e-01,
        6.40228271e-01,  5.36144257e-01,  5.90462744e-01,  5.66305816e-01,
        5.77841461e-01,  6.21604323e-01,  6.85790598e-01,  6.51344478e-01,
        4.33390116e-04, 

In [186]:
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [187]:
model.fit(x_train,y_train)

MLPClassifier(activation='relu', alpha=0.01, batch_size=256, beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(300,), learning_rate='adaptive',
              learning_rate_init=0.001, max_fun=15000, max_iter=500,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [189]:
import pickle
# Save the Modle to file in the current working directory
#For any new testing data other than the data in dataset

Pkl_Filename = "Emotion_Voice_Detection_Model.pkl"  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(model, file)

In [190]:
with open(Pkl_Filename, 'rb') as file:  
    Emotion_Voice_Detection_Model = pickle.load(file)

Emotion_Voice_Detection_Model

MLPClassifier(activation='relu', alpha=0.01, batch_size=256, beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(300,), learning_rate='adaptive',
              learning_rate_init=0.001, max_fun=15000, max_iter=500,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [192]:
y_pred=Emotion_Voice_Detection_Model.predict(x_test)
y_pred

array(['calm', 'fearful', 'happy', 'calm', 'calm', 'calm', 'disgust',
       'calm', 'calm', 'fearful', 'fearful', 'disgust', 'calm', 'calm',
       'happy', 'calm', 'calm', 'happy', 'disgust', 'fearful', 'calm',
       'happy', 'fearful', 'happy', 'disgust', 'disgust', 'disgust',
       'happy', 'happy', 'calm', 'happy', 'fearful', 'happy', 'disgust',
       'disgust', 'disgust', 'fearful', 'calm', 'calm', 'fearful',
       'happy', 'disgust', 'fearful', 'fearful', 'calm', 'calm',
       'disgust', 'fearful', 'disgust', 'fearful', 'fearful', 'disgust',
       'fearful', 'calm', 'fearful', 'happy', 'disgust', 'disgust',
       'happy', 'fearful', 'fearful', 'fearful', 'disgust', 'happy',
       'calm', 'happy', 'disgust', 'fearful', 'fearful', 'happy',
       'fearful', 'disgust', 'happy', 'fearful', 'calm', 'disgust',
       'happy', 'fearful', 'happy', 'disgust', 'happy', 'calm', 'fearful',
       'disgust', 'calm', 'disgust', 'happy', 'calm', 'happy', 'disgust',
       'happy', 'fea

In [193]:
import numpy as np
import pandas as pd
y_pred1 = pd.DataFrame(y_pred, columns=['predictions'])
y_pred1['file_names'] = test_filename
print(y_pred1)
y_pred1.to_csv('predictionfinal.csv')

    predictions                file_names
0          calm  03-01-02-01-01-02-12.wav
1       fearful  03-01-06-01-01-02-22.wav
2         happy  03-01-03-01-02-02-04.wav
3          calm  03-01-02-01-02-02-21.wav
4          calm  03-01-02-01-02-01-04.wav
..          ...                       ...
187     fearful  03-01-06-01-02-02-04.wav
188        calm  03-01-02-01-01-02-05.wav
189        calm  03-01-02-01-01-02-10.wav
190        calm  03-01-02-02-01-01-13.wav
191     fearful  03-01-06-02-02-01-19.wav

[192 rows x 2 columns]


In [196]:
y_pred=model.predict(x_test)

In [197]:
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

In [198]:
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 81.77%
